# **Jour 44 : USArrests**

Le jeu de données **USArrests** est un ensemble de statistiques classiques qui fournit des informations sur la criminalité et le nombre d'arrestations dans chacun des **50** États américains en **1973**.

## 0. Chargement des librairies 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import plotly.express as px
import statsmodels.api as sm


## 1. Chargement de la base données

In [10]:
data = pd.read_csv("../data/USArrests.csv")


In [11]:
data.head()

,Unnamed: 0,Murder,Assault,UrbanPop,Rape
0,Alabama,13.2,236,58,21.2
1,Alaska,10.0,263,48,44.5
2,Arizona,8.1,294,80,31.0
3,Arkansas,8.8,190,50,19.5
4,California,9.0,276,91,40.6


### 1.2. Vérification des valeurs manquantes

In [13]:
print(data.isnull().sum())

Unnamed: 0    0
Murder        0
Assault       0
UrbanPop      0
Rape          0
dtype: int64


Aucune valeur manquante ne se trouve dans la base.